# Integrating GPU libraries for fun and profit

### ...on extending and interfacing HPC simulation tools

##### Authors: Adrian Oeftiger and Martin Schwinzerl

# Abstract

I have a high-performance number crunching tool with cool physics which simulates long-term on a GPU &ndash; how can I extend the inner loop by further cool physics, injected from the outside?
In python this should be easy, right? But wait... we are sitting on device memory?

In this talk we explore how to tightly couple two libraries for high-performance computation of long-term beam dynamics, `SixTrackLib` and `PyHEADTAIL`. How can we design the _interface_ between both libraries in terms of

(1) remaining on the python level,

(2) avoid losing performance due to device-to-host-to-device copies, and

(3) keeping both libraries as stand-alone packages?

The interface can be surprisingly simple, yet fully fledged... Let's go!

# ... the Physics ...

# Collective beam dynamics

3D particle motion $\leadsto$ 6 phase space coordinates:
$$\mathbb{X}=(\underbrace{x, x'\vphantom{y'}}_{horizontal}, \underbrace{y, y'}_{vertical}, \underbrace{z, \delta\vphantom{y'}}_{longitudinal})$$

A beam $=$ state of $N$ macro-particles $=$ $6N$ values of phase space coordinates

## Simulations:
* typically up to $\mathcal{O}(10^6)$ macro-particles
* accelerator elements to track through: up to $\mathcal{O}(1000)$
* simulations can last up to $\mathcal{O}(10^6)$ turns 
* particle-to-particle interaction: binning, FFT, convolution, particle-in-cell, Poisson solvers

# Requirements for simulation tools

* long-term evolution $\leadsto$ **double precision**
* heavy **number crunching** $\leadsto$ high-performance computing (**HPC**)<br />(in particular for _collective effects_ i.e. particle-to-particle interaction)
* **iterative development**, frequent update of accelerator models $\leadsto$ python

# Single-particle vs. multi-particle dynamics

**single-particle** $\implies$ <span style="color:#00aad4;">"tracking"</span>: particle motion due to external focusing (magnets and RF cavities)

**multi-particle** $\implies$ <span style="color:#aa0000;">"collective effect kicks"</span>: direct and indirect particle-to-particle interaction

<center>
<img src="images/transverse-model_empty.png" alt="sketch of one-turn map for ring" style="width:30%;"/>
Tracking around the accelerator ring
</center>

# The `PyHEADTAIL` library

Python based code for simulating collective beam dynamics: [github repo $\nearrow$](https://github.com/PyCOMPLETE/PyHEADTAIL)

$\implies$ simplified matrix-based <span style="color:#00aad4;">tracking</span>

$\implies$ **strong:** detailed models for <span style="color:#aa0000;">collective effect kicks</span>

<center>
<img src="images/wakefield.png" alt="wake field sketch" style="width:30%;"/>
Example for a kick: wakefield induced by leading particles imparting kicks on trailing particles
</center>

# The `SixTrackLib` library

C templated code with Python API for simulating single-particle beam dynamics: [github repo $\nearrow$](https://github.com/sixtrack/sixtracklib)

$\implies$ **strong:** advanced non-linear <span style="color:#00aad4;">tracking</span>

$\implies$ approximative / simplified models for <span style="color:#aa0000;">collective effect kicks</span>

<center>
<img src="images/tracking.png" alt="tracking sketch" style="width:30%;"/>
Example for tracking: <span style="color:#ff6701;">dipole magnets bend</span> and <span style="color:#800000;">quadrupole magnets (de-)</span><span style="color:#000080;">focus</span> particles
</center>

# ... the HPC part ...

# `PyHEADTAIL` on the GPU

[Concept presented on PyHEP'19 $\nearrow$](https://indico.cern.ch/event/833895/contributions/3577801/), in short:

<div style="float: right; width: 10%">
<img src="images/duck.png" alt="duck typing" style="width: 100%;" /> <br />
<!-- <img src="pycuda-logo2.png" alt="PyCUDA logo" style="width: 100%;" /> -->
</div>

* utilise **duck typing** to separate physics from back-end implementation
* sandwich layer via **context management** and **function redirection**<br />(math dictionaries)
* exploit GPU via `PyCUDA` (`CuPy` would work similarly)

In [ ]:
# tracking loop in PyHEADTAIL:
with GPU(pyht_beam):
    for i in range(n_turns):
        for element in pyht_ring_elements:
            element.track(pyht_beam)

$\implies$ implement physics only once! <br />
$\implies$ back-end details transparent to users / high-level developers

# `SixTrackLib` on the GPU

[Concept presented on PyHEP'19 $\nearrow$](https://indico.cern.ch/event/833895/contributions/3577803/), in short:

* Python API for dynamic interaction
* C **templating approach** to separate physics from back-end implementation
    * implementation in usual C for (single-core) CPU
    * implementation in openCL for multi-core CPU and GPU (AMD, NVIDIA)
    * implementation in CUDA for NVIDIA GPUs

In [ ]:
# tracking kernel in SixTrackLib
trackjob = stl.TrackJob(stl_ring_elements, stl_beam, device='opencl:0.0')
trackjob.track_until(n_turns)

$\implies$ implement physics only once! <br />
$\implies$ users launch "trackjobs" with just a single `device` keyword to switch architecture

# ... the quest ...

# `SixTrackLib` $+$ `PyHEADTAIL` $=$ <3 ?

`SixTrackLib` $\rightarrow$ strong in advanced non-linear <span style="color:#00aad4;">tracking</span>

`PyHEADTAIL` $\rightarrow$ strong in <span style="color:#aa0000;">collective effect kicks</span>

$\implies$ `SixTrackLib` $+$ `PyHEADTAIL` $=$ strong in both?

<center>
<img src="images/transverse-model_integration.png" alt="integrating PyHEADTAIL and SixTrackLib" style="width:30%;"/>
Alternating SixTrackLib <span style="color:#00aad4;">tracking</span> and PyHEADTAIL <span style="color:#aa0000;">kicking</span>
</center>

# ... the quest*ions* ...

Situation:
* `PyHEADTAIL` on the GPU lives in `PyCUDA`, managed from user script
* `SixTrackLib` on the GPU manages its CUDA kernel invocation by itself
* beam state data: <br />`PyHEADTAIL` uses `PyCUDA.GPUArrays` while `SixTrackLib` uses custom memory buffers

Critical aspects:
* how to avoid GPU - CPU - GPU memory transfer?
* how to hand over the data between the two codes?
* can `PyHEADTAIL`'s `PyCUDA` context interact with `SixTrackLib`'s initialised context? 
    * visibility of memory to each other
    * alternating kernel calls?
    * flow control from python level?

# ... well &ndash; let's try to unify, shan't we?

Importing SixTrackLib:

In [1]:
import sixtracklib as stl

# print current commit
import os
stl_dir = os.path.dirname(stl.__file__)
!printf "SixTrackLib " && cd $stl_dir && git log | head -1

SixTrackLib commit f42c420da20602c9c4c08172338ed3019bb20d25


Importing PyHEADTAIL:

In [2]:
import sys
sys.path = ['/home/oeftiger/gsi/git/PyHEADTAIL/'] + sys.path

import PyHEADTAIL

PyHEADTAIL v1.14.1.7


